In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import html
import scrapy
from time import sleep
import urllib3
import json
from selenium import webdriver
import random

In [2]:
def parser(link):

    encabezados = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
    }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text

    #soup = get_Soup('https://minciencias.gov.co/convocatorias/todas')
    parser = html.fromstring(resp)
    urllib3.disable_warnings()

    return parser

In [3]:
link = 'http://www.fapemig.br/pt/chamadas_encerradas_oportunidades_fapemig/'

In [4]:
parserT = parser(link)

titles_brutos = parserT.xpath('//h1//text()')
titles = []
for t in range(len(titles_brutos)):
    if not t % 2:
        titles.append(titles_brutos[t].strip() + ' ' + titles_brutos[t+1].strip())
        


In [5]:
ids_brutos = parserT.xpath('//a/@href')
ids_proyectos = []
for i in ids_brutos:
    if i.startswith('#') and i != '#':
        ids_proyectos.append(i.strip('#'))

len(ids_proyectos)

78

In [6]:
pdfs_proyectos = []
pdf = ''
for i in ids_proyectos:
    pdfs_proyecto = parserT.xpath('//div[@id="' + i + '"]//a/@href')
    pdfs = ''
    for p in pdfs_proyecto:
        p = 'http://www.fapemig.br/' + p.strip('../../')
        if p.endswith('.pdf'):
            pdfs = pdfs + p + ', '
    
    pdfs_proyectos.append(pdfs.strip(', '))

len(pdfs_proyectos)

78

In [7]:
fechas = parserT.xpath('//p[contains(text(), "Data de encerrament")]//text()')

In [8]:
link = 'http://www.fapemig.br/pt/chamadas_encerradas_oportunidades_fapemig/'

fechas_apertura = []
fechas_cierre = []
descripciones_proyectos = []

driver = webdriver.Firefox()
driver.get(link)

clicks = driver.find_elements_by_xpath('//h5')

for c in clicks:

    c.click()

    descripcion_proyecto_bruto = driver.find_elements_by_xpath('//div[@class="row mt-1"]/div//p')

    descripcion_proyecto = ''

    for desc in descripcion_proyecto_bruto:

        descripcion_proyecto = descripcion_proyecto + desc.text + ' '

    descripciones_proyectos.append(descripcion_proyecto.strip())


    try:
        if 'inclusão' or 'Inclusão:' in descripcion_proyecto:
            descripcion_proyecto = descripcion_proyecto.lower()
            fechas_apertura.append(descripcion_proyecto.split('inclusão:')[1].strip().split(' ')[0])
    except:
        fechas_apertura.append('')



    try:
        if 'encerramento' or 'Encerramento:' in descripcion_proyecto:
            descripcion_proyecto = descripcion_proyecto.lower()
            fechas_cierre.append(descripcion_proyecto.split('encerramento:')[1].strip().split(' ')[0])

    except:
        fechas_cierre.append('')

    c.click()

driver.quit()

In [9]:
fapemig = pd.DataFrame()

fapemig['Título'] = titles
fapemig['Descripción'] = descripciones_proyectos
fapemig['Fecha Apertura'] = fechas_apertura
fapemig['Fecha Cierre'] = fechas_cierre
fapemig['Pdfs'] = pdfs_proyectos


In [11]:
fapemig.to_excel('brasil_fapemig.xlsx')